This notebook computes all of the "distances" (the fuzzy logic scores for IOU, DICE, ...) between the GradCam maps produced by the final models and the visual characterstics maps created by the dermatologists. That is: 

```
for each image i  
  for each GradCam image i_gc  
    for each dermatologist d  
      for each characteristic d_char  
        compute the visual distance between i_gc and d_char 
```


In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from pathlib import Path
from collections import defaultdict
import matplotlib.pyplot as plt 
from PIL import Image
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [2]:
classes = [
    "Acne",
    "Actinic keratosis",
    "Psoriasis",
    "Seborrheic dermatitis",
    "Viral warts",
    "Vitiligo"
]

derms = [
    'derm1',
    'derm2',
    'derm3',
    'derm4',
    'derm5',
    'derm6',
    'derm7',
    'derm8',
]

characteristics = [
    'Closed comedo',
    'Cyst',
    'Dermatoglyph disruption',
    'Leukotrichia',
    'Macule',
    'Nodule',
    'Open comedo',
    'Papule',
    'Patch',
    'Plaque',
    'Pustule',
    'Scale',
    'Scar',
    'Sun damage',
    'Telangiectasia',
    'Thrombosed capillaries'
]

images_path = Path('/home/ubuntu/hot-store/dermx_finetuning/split1/test')
results_path = Path('/home/ubuntu/hot-store/final/')

# Metrics supporting probabilistic segmentation maps
fuzzy_and = lambda x,y: np.minimum(x,y)
fuzzy_or = lambda x,y: np.maximum(x,y)
fuzzy_not = lambda x: 1-x

def pixel_metrics_fuzzy(y_true, y_pred):
    """
    Pixel-level metrics of segmentation accuracy following fuzzy logic operators.
    
    :param y_true: numpy.ndarray of reference segmentation, values in [0,1]
    :param y_pred: numpy.ndarray of predicted segmentation, values in [0,1]

    :return: a dictionary encoding the metrics
    """
        
    np.testing.assert_equal(y_true.shape, y_pred.shape, err_msg="Expecting \
    the reference and predicted segmentations to be of the same size.")
    
    # Check the ranges
    np.testing.assert_equal(np.logical_and(y_true >= 0, y_true <= 1).all(), True, err_msg="Expecting \
    the reference segmentations to be in the range 0 to 1.")
    np.testing.assert_equal(np.logical_and(y_pred >= 0, y_pred <= 1).all(), True, err_msg="Expecting \
    the predicted segmentations to be in the range 0 to 1.")
    
    TP = fuzzy_and(y_true, y_pred).sum()
    TN = fuzzy_and(fuzzy_not(y_true), fuzzy_not(y_pred)).sum()
    union = fuzzy_or(y_true, y_pred).sum()
    
    metrics = {}
    
    # Summary metrics
    metrics["iou"] = TP / union
    metrics["dice"] = 2 * TP / ( y_true.sum() + y_pred.sum() ) 
    
    # Positive class metrics
    metrics["precision"] = TP / y_pred.sum()
    metrics["recall"] = TP / y_true.sum()
    
    # Negative class metrics
    metrics["negative_predictive_value"] = TN / fuzzy_not(y_pred).sum()
    metrics["specificity"] = TN / fuzzy_not(y_true).sum()
    
    return metrics


def calculate_mask_metrics(gradcam_path, mask_path, interpolation_method=Image.BICUBIC):
    """
    Calculates the fuzzy logic metrics given the paths to a pair of input images.
    The derm mask is resized to match the size of the gradcam image.
    
    Input:
    - gradcam_image_path: Pathlib path to a gradCam image. The file is assumed to be in .npy format.
    - derm_char_mask_path: Pathlib path to a derm annotation. The file is assumed to be in a format that can
                           be opened by PIL.
    - interpolation_method: String. The method used for interpolation when resizing the derm mask. Options are
                            NEAREST, BOX, BILINEAR, HAMMING, BICUBIC, LANCZOS. Default is NEAREST.    
    """
    
    # Open images.
    gradcam = Image.fromarray(np.load(gradcam_path, allow_pickle=True))
    mask = np.asarray(Image.open(mask_path)) / 255
    
    # Resize the derm mask if its size does not match the size of the gradcam image.
    if gradcam.size[::-1] != mask.shape:
        # Note that resize uses (cols, rows) format, while .shape is in (rows, cols) format
        gradcam = gradcam.resize((mask.shape[1], mask.shape[0]), interpolation_method)

    # Normalize to 0-1
    gradcam = np.asarray(gradcam)
    if gradcam.min() != gradcam.max():
        gradcam = (gradcam - gradcam.min()) / (gradcam.max() - gradcam.min())
    
    return pixel_metrics_fuzzy(mask, gradcam)


## Per image metrics

In [5]:
def get_per_image_metrics(results_path, model_name, images_path, masks_path, classes, columns):
    gradcams_path = results_path / 'visualisation/gradcam' / model_name
    model_path = results_path / model_name
    
    derm_mask_paths = [mask_path for mask_path in masks_path.iterdir() if mask_path.suffix == '.png']
    image_paths = [image_path for image_path in images_path.rglob('*.jpeg')]

    # For each image in the test set, calculate the value of the defined metrics given the GradCam image for a given
    # class and the outline made by a specific derm for a given characteristic. The result is stored in a defaultdict.
    # As a sanity check, the number of matches between GradCam images and derm annotations is also calculated.
    metrics_list = []

    # Set the method used for interpolation when resizing the derm annotations.
    interpolation = Image.BICUBIC

    # Compute metrics for each image
    for image_path in image_paths:
        for diagnosis in classes:
            # Check if matching GradCam file exists.
            gradcam_path = gradcams_path / Path(image_path.stem + '_' + diagnosis + '.npy')
            if gradcam_path.is_file():
                mask_path = masks_path / Path(image_path.stem + '.png') 
                if mask_path.is_file():
                    # Calculate the value of the metics given the GradCam image and the derm mask.
                    gradcam_metric_val = calculate_mask_metrics(gradcam_path, mask_path, interpolation)
                    gradcam_metric_val['gradcam_class'] = diagnosis 
                    gradcam_metric_val['filename'] = image_path.stem 
                    metrics_list.append(gradcam_metric_val)
            else:
                print(f'GradCam file missing for image {image_path}')
                
    return pd.DataFrame.from_records(metrics_list, columns=columns)


In [6]:
def analyse_per_image_gradcam(
    results_path, 
    model_name, 
    images_path, 
    masks_path, 
    preds_name,
    columns,
    classes,
):
    per_image_metrics_df = get_per_image_metrics(results_path, model_name, images_path, masks_path, classes, columns)
    
    preds_df = pd.read_csv(results_path / preds_name)
    preds_df['filename'] = [filename.split('/')[1].split('.')[0] for filename in preds_df.filename.values]
    preds_df = preds_df.merge(per_image_metrics_df, left_on='filename', right_on='filename')
    # Keep only correct predictions
    preds_df = preds_df[preds_df['actual'] == preds_df['pred']]
    preds_df = preds_df[preds_df['pred_class'] == preds_df['gradcam_class']]
    
    return {
        'iou': preds_df['iou'].mean(),
        'dice': preds_df['dice'].mean(),
        'precision': preds_df['precision'].mean(),
        'recall': preds_df['recall'].mean(),
        'negative_predictive_value': preds_df['negative_predictive_value'].mean(),
        'specificity': preds_df['specificity'].mean(), 
    }


In [8]:
model_info = [
    {
        'model_name': 'efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_',
        'model_architecture': 'efficientnet',
    },
    {
        'model_name': 'inception_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.001_lactivation_85_',
        'model_architecture': 'inception',
    },
    {
        'model_name': 'inceptionresnetv2_r20_s0.25_z0.5_b[0.75, 1.25]_lr0.0001_lblock8_9_ac_',
        'model_architecture': 'inceptionresnet',
    },
    {
        'model_name': 'mobilenetv1_r10_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv_pw_12_relu_',
        'model_architecture': 'mobilenetv1',
    },
    {
        'model_name': 'mobilenetv2_r10_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock_15_add_',
        'model_architecture': 'mobilenetv2',
    },
    {
        'model_name': 'nasnetmobile_r20_s0.25_z0.5_b[0.5, 1.0]_lr0.0001_lnormal_concat_11_',
        'model_architecture': 'nasnetmobile',
    },
    {
        'model_name': 'resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_',
        'model_architecture': 'resnet',
    },
    {
        'model_name': 'resnetv2_r20_s0.25_z0.25_b[0.5, 1.0]_lr0.001_lpost_relu_',
        'model_architecture': 'resnetv2',
    },
    {
        'model_name': 'vgg_r10_s0.0_z0.25_b[0.5, 1.0]_lr0.01_lblock5_pool_',
        'model_architecture': 'vgg',
    },
    {
        'model_name': 'xception_r10_s0.25_z0.5_b[0.5, 1.5]_lr0.001_lblock14_sepconv2_act_', 
        'model_architecture': 'xception',
    }
]
masks_path = Path('/home/ubuntu/hot-store/dermx_masks/per_image')
columns = [
         'filename',
         'gradcam_class',
         'iou',
         'dice',
         'precision',
         'recall',
         'negative_predictive_value',
         'specificity'
    ]

models_performance = []
for model in model_info:
    for idx in range(5):
        model_results_path = results_path / model['model_architecture']
        model_name = f'{model["model_name"]}{idx}_finetuned'
        preds_name = f'{model["model_name"]}{idx}_finetuned_preds.csv'
        print(model_name)
        model_performance = analyse_per_image_gradcam(model_results_path, model_name, images_path, masks_path, preds_name, columns, classes)
        model_performance['model_architecture'] = model['model_architecture']
        model_performance['model_name'] = model['model_name']
        models_performance.append(model_performance)
per_image_models_performance_df = pd.DataFrame.from_records(models_performance, columns=models_performance[0].keys())   

efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_0_finetuned


/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: invalid value encountered in double_scalars


efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_1_finetuned


/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: invalid value encountered in double_scalars


efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_2_finetuned


...

/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: invalid value encountered in double_scalars


xception_r10_s0.25_z0.5_b[0.5, 1.5]_lr0.001_lblock14_sepconv2_act_3_finetuned


/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: invalid value encountered in double_scalars


xception_r10_s0.25_z0.5_b[0.5, 1.5]_lr0.001_lblock14_sepconv2_act_4_finetuned


/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: invalid value encountered in double_scalars


In [15]:
per_image_models_performance_df.groupby('model_architecture').describe().to_csv('per_image_models_performance.csv')

In [16]:
per_image_models_performance_df.groupby('model_architecture').describe()

iou                                                    \
                   count      mean       std       min       25%       50%   
model_architecture                                                           
efficientnet         5.0  0.235493  0.005021  0.228460  0.234184  0.234975   
inception            5.0  0.251345  0.005117  0.244788  0.247225  0.252841   
inceptionresnet      5.0  0.228990  0.005046  0.223323  0.226216  0.226861   
mobilenetv1          5.0  0.220388  0.005437  0.211690  0.218448  0.223120   
mobilenetv2          5.0  0.210745  0.010535  0.198710  0.201290  0.212890   
nasnetmobile         5.0  0.255228  0.013873  0.237593  0.244550  0.259659   
resnet               5.0  0.189707  0.004265  0.183386  0.188164  0.189847   
resnetv2             5.0  0.215654  0.008872  0.204736  0.207551  0.219498   
vgg                  5.0  0.190755  0.011459  0.179209  0.183470  0.189110   
xception             5.0  0.275057  0.001976  0.272352  0.274056  0.275301   

                                        dice                                \
                         75%       max count      mean       std       min   
model_architecture                                                           
efficientnet        0.237674  0.242173   5.0  0.358810  0.007004  0.349435   
inception           0.255508  0.256363   5.0  0.382734  0.006266  0.375425   
inceptionresnet     0.233376  0.235177   5.0  0.355706  0.006795  0.347582   
mobilenetv1         0.224102  0.224582   5.0  0.344956  0.006618  0.334541   
mobilenetv2         0.217423  0.223414   5.0  0.328587  0.014821  0.312508   
nasnetmobile        0.262694  0.271643   5.0  0.387520  0.017416  0.365485   
resnet              0.193039  0.194101   5.0  0.304816  0.006601  0.295402   
resnetv2            0.223014  0.223469   5.0  0.340214  0.012727  0.324101   
vgg                 0.193041  0.208946   5.0  0.303165  0.015001  0.288138   
xception            0.276000  0.277576   5.0  0.414856  0.002145  0.412145   

                                                           precision  \
                         25%       50%       75%       max     count   
model_architecture                                                     
efficientnet        0.356489  0.357851  0.361823  0.368450       5.0   
inception           0.377790  0.382583  0.387888  0.389985       5.0   
inceptionresnet     0.352518  0.353002  0.361477  0.363949       5.0   
mobilenetv1         0.342381  0.347876  0.349329  0.350652       5.0   
mobilenetv2         0.314074  0.332318  0.337847  0.346190       5.0   
nasnetmobile        0.373679  0.394528  0.396021  0.407886       5.0   
resnet              0.302055  0.304727  0.309762  0.312136       5.0   
resnetv2            0.329224  0.345359  0.351090  0.351294       5.0   
vgg                 0.294731  0.299499  0.306196  0.327260       5.0   
xception            0.413793  0.414926  0.415472  0.417945       5.0   

                                                                      \
                        mean       std       min       25%       50%   
model_architecture                                                     
efficientnet        0.351102  0.007854  0.342810  0.345047  0.351736   
inception           0.355432  0.008211  0.346446  0.347426  0.357427   
inceptionresnet     0.369433  0.012082  0.353382  0.368184  0.368962   
mobilenetv1         0.319327  0.009876  0.305103  0.313821  0.323485   
mobilenetv2         0.306810  0.021252  0.284123  0.285279  0.311602   
nasnetmobile        0.346991  0.022127  0.317802  0.330280  0.354922   
resnet              0.323528  0.008538  0.310174  0.322114  0.323391   
resnetv2            0.410362  0.018763  0.383507  0.398465  0.419915   
vgg                 0.334374  0.007345  0.323508  0.330174  0.337969   
xception            0.443366  0.007993  0.429435  0.444976  0.445814   

                                       recall                                \
                         75%       max

## Per characteristic metrics

In [11]:
def get_per_characteristic_metrics(results_path, model_name, images_path, masks_path, classes, columns, characteristics):
    gradcams_path = results_path / 'visualisation/gradcam' / model_name
    model_path = results_path / model_name
    
    derm_mask_paths = [mask_path for mask_path in masks_path.iterdir() if mask_path.suffix == '.png']
    image_paths = [image_path for image_path in images_path.rglob('*.jpeg')]

    # For each image in the test set, calculate the value of the defined metrics given the GradCam image for a given
    # class and the outline made by a specific derm for a given characteristic. The result is stored in a defaultdict.
    # As a sanity check, the number of matches between GradCam images and derm annotations is also calculated.
    metrics_list = []

    # Set the method used for interpolation when resizing the derm annotations.
    interpolation = Image.NEAREST

    # Compute metrics for each image
    for image_path in image_paths:
        for characteristic in characteristics:
            for diagnosis in classes:
                # Check if matching GradCam file exists.
                gradcam_path = gradcams_path / Path(image_path.stem + '_' + diagnosis + '.npy')
                if gradcam_path.is_file():
                    mask_path = masks_path / Path(f'{characteristic}_{image_path.stem}.png') 
                    if mask_path.is_file():
                        # Calculate the value of the metics given the GradCam image and the derm mask.
                        gradcam_metric_val = calculate_mask_metrics(gradcam_path, mask_path, interpolation)
                        gradcam_metric_val['gradcam_class'] = diagnosis 
                        gradcam_metric_val['characteristic'] = characteristic 
                        gradcam_metric_val['filename'] = image_path.stem 
                        metrics_list.append(gradcam_metric_val)
                else:
                    print(f'GradCam file missing for image {image_path}')
                
    return pd.DataFrame.from_records(metrics_list, columns=columns)


In [12]:
def analyse_per_characteristics_gradcam(
    results_path, 
    model_name, 
    images_path, 
    masks_path, 
    preds_name,
    columns,
    classes,
    characteristics,
    metrics
):
    per_image_metrics_df = get_per_characteristic_metrics(results_path, model_name, images_path, masks_path, classes, columns, characteristics)
    preds_df = pd.read_csv(results_path / preds_name)
    preds_df['filename'] = [filename.split('/')[1].split('.')[0] for filename in preds_df.filename.values]
    preds_df = preds_df.merge(per_image_metrics_df, left_on='filename', right_on='filename')
    # Keep only correct predictions
    preds_df = preds_df[preds_df['actual'] == preds_df['pred']]
    preds_df = preds_df[preds_df['pred_class'] == preds_df['gradcam_class']]

    metrics_dict = {}
    for characteristic in characteristics:
        metrics_dict[characteristic] = {}
        for metric in metrics:
            metrics_dict[characteristic][metric] = preds_df[preds_df['characteristic'] == characteristic][metric].mean()
            
    return metrics_dict


In [14]:
model_info = [
    {
        'model_name': 'efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_',
        'model_architecture': 'efficientnet',
    },
    {
        'model_name': 'inception_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.001_lactivation_85_',
        'model_architecture': 'inception',
    },
    {
        'model_name': 'inceptionresnetv2_r20_s0.25_z0.5_b[0.75, 1.25]_lr0.0001_lblock8_9_ac_',
        'model_architecture': 'inceptionresnet',
    },
    {
        'model_name': 'mobilenetv1_r10_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv_pw_12_relu_',
        'model_architecture': 'mobilenetv1',
    },
    {
        'model_name': 'mobilenetv2_r10_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock_15_add_',
        'model_architecture': 'mobilenetv2',
    },
    {
        'model_name': 'nasnetmobile_r20_s0.25_z0.5_b[0.5, 1.0]_lr0.0001_lnormal_concat_11_',
        'model_architecture': 'nasnetmobile',
    },
    {
        'model_name': 'resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_',
        'model_architecture': 'resnet',
    },
    {
        'model_name': 'resnetv2_r20_s0.25_z0.25_b[0.5, 1.0]_lr0.001_lpost_relu_',
        'model_architecture': 'resnetv2',
    },
    {
        'model_name': 'vgg_r10_s0.0_z0.25_b[0.5, 1.0]_lr0.01_lblock5_pool_',
        'model_architecture': 'vgg',
    },
    {
        'model_name': 'xception_r10_s0.25_z0.5_b[0.5, 1.5]_lr0.001_lblock14_sepconv2_act_', 
        'model_architecture': 'xception',
    }
]
masks_path = Path('/home/ubuntu/hot-store/dermx_masks/per_characteristic')
columns = [
    'filename',
    'gradcam_class',
    'characteristic',
    'iou',
    'dice',
    'precision',
    'recall',
    'negative_predictive_value',
    'specificity'
]
metrics = [
    'iou',
    'dice',
    'precision',
    'recall',
    'negative_predictive_value',
    'specificity'
]

models_performance_dict = {}
for model in model_info:
    for idx in range(5):
        model_results_path = results_path / model['model_architecture']
        model_name = f'{model["model_name"]}{idx}_finetuned'
        preds_name = f'{model["model_name"]}{idx}_finetuned_preds.csv'
        print(model_name)
        model_performance = analyse_per_characteristics_gradcam(model_results_path, model_name, images_path, masks_path, preds_name, columns, classes, characteristics, metrics)
        for k, v in model_performance.items():
            models_performance_dict[(model['model_architecture'], model_name, k)] = v

per_characteristic_models_performance_df = pd.DataFrame(models_performance_dict).T

efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_0_finetuned


/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: invalid value encountered in double_scalars


efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_1_finetuned


/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: invalid value encountered in double_scalars


efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_2_finetuned


...

/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: invalid value encountered in double_scalars


xception_r10_s0.25_z0.5_b[0.5, 1.5]_lr0.001_lblock14_sepconv2_act_3_finetuned


/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: invalid value encountered in double_scalars


xception_r10_s0.25_z0.5_b[0.5, 1.5]_lr0.001_lblock14_sepconv2_act_4_finetuned


/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: invalid value encountered in double_scalars


In [17]:
per_characteristic_models_performance_df.reset_index().groupby(['level_0', 'level_2']).describe().to_csv('per_characteristic_models_performance.csv')

In [28]:
characteristics_results_df = per_characteristic_models_performance_df.reset_index().groupby(['level_0', 'level_2']).describe()


iou                                          \
                        count      mean       std       min       25%   
level_2                                                                 
Closed comedo             5.0  0.132949  0.009129  0.118577  0.130097   
Cyst                      5.0  0.017063  0.005245  0.011614  0.012280   
Dermatoglyph disruption   5.0  0.082566  0.006857  0.074834  0.078839   
Leukotrichia              5.0  0.104993  0.019929  0.075889  0.094343   
Macule                    5.0  0.088351  0.003963  0.085517  0.085866   
Nodule                    5.0  0.043468  0.005063  0.038597  0.040814   
Open comedo               5.0  0.117793  0.002078  0.115260  0.117153   
Papule                    5.0  0.109408  0.004649  0.104323  0.106931   
Patch                     5.0  0.173787  0.011094  0.156641  0.170866   
Plaque                    5.0  0.149935  0.002498  0.147399  0.149233   
Pustule                   5.0  0.028107  0.000817  0.027560  0.027696   
Scale                     5.0  0.162367  0.004414  0.156543  0.160029   
Scar                      5.0  0.212884  0.006961  0.200877  0.214219   
Sun damage                5.0  0.272498  0.041998  0.204328  0.272771   
Telangiectasia            5.0  0.072952  0.008863  0.060246  0.068093   
Thrombosed capillaries    5.0  0.020479  0.002378  0.018058  0.018344   

                                                       dice            \
                              50%       75%       max count      mean   
level_2                                                                 
Closed comedo            0.135132  0.139845  0.141093   5.0  0.199618   
Cyst                     0.018098  0.019006  0.024317   5.0  0.032691   
Dermatoglyph disruption  0.082563  0.083359  0.093233   5.0  0.145974   
Leukotrichia             0.110302  0.120384  0.124049   5.0  0.183608   
Macule                   0.087011  0.088169  0.095192   5.0  0.143627   
Nodule                   0.042670  0.043358  0.051902   5.0  0.078789   
Open comedo              0.117241  0.118392  0.120920   5.0  0.179550   
Papule                   0.108179  0.111128  0.116478   5.0  0.169515   
Patch                    0.174512  0.182391  0.184523   5.0  0.282444   
Plaque                   0.149239  0.149689  0.154116   5.0  0.245440   
Pustule                  0.027823  0.027907  0.029549   5.0  0.048734   
Scale                    0.162750  0.164239  0.168276   5.0  0.256025   
Scar                     0.214322  0.216241  0.218759   5.0  0.324956   
Sun damage               0.280243  0.286157  0.318989   5.0  0.414811   
Telangiectasia           0.074653  0.080417  0.081352   5.0  0.133803   
Thrombosed capillaries   0.020163  0.022579  0.023252   5.0  0.039483   

                                                                           \
                              std       min       25%       50%       75%   
level_2                                                                     
Closed comedo            0.012123  0.181100  0.194454  0.202998  0.209130   
Cyst                     0.009523  0.022671  0.023950  0.034884  0.036292   
Dermatoglyph disruption  0.010182  0.134500  0.139976  0.145825  0.148012   
Leukotrichia             0.031600  0.138033  0.165875  0.191824  0.208202   
Macule                   0.007018  0.139085  0.139613  0.141202  0.142259   
Nodule                   0.007289  0.071207  0.074804  0.077886  0.079523   
Open comedo              0.003756  0.174322  0.178866  0.179183  0.180593   
Papule                   0.006836  0.161961  0.165188  0.168535  0.172238   
Patch                    0.017119  0.256372  0.278001  0.282380  0.296118   
Plaque                   0.003701  0.241613  0.244605  0.244665  0.244693   
Pustule                  0.000878  0.048056  0.048259  0.048462  0.048639   
Scale                    0.007717  0.246637  0.251757  0.256341  0.258070   
Scar                     0.009964  0.308200  0.325204  0.326655  0.330986   
Sun damage        

In [31]:
characteristics_results_df

iou                                \
                                        count      mean       std       min   
level_0         level_2                                                       
efficientnet    Closed comedo             5.0  0.132949  0.009129  0.118577   
                Cyst                      5.0  0.017063  0.005245  0.011614   
                Dermatoglyph disruption   5.0  0.082566  0.006857  0.074834   
                Leukotrichia              5.0  0.104993  0.019929  0.075889   
                Macule                    5.0  0.088351  0.003963  0.085517   
                Nodule                    5.0  0.043468  0.005063  0.038597   
                Open comedo               5.0  0.117793  0.002078  0.115260   
                Papule                    5.0  0.109408  0.004649  0.104323   
                Patch                     5.0  0.173787  0.011094  0.156641   
                Plaque                    5.0  0.149935  0.002498  0.147399   
                Pustule                   5.0  0.028107  0.000817  0.027560   
                Scale                     5.0  0.162367  0.004414  0.156543   
                Scar                      5.0  0.212884  0.006961  0.200877   
                Sun damage                5.0  0.272498  0.041998  0.204328   
                Telangiectasia            5.0  0.072952  0.008863  0.060246   
                Thrombosed capillaries    5.0  0.020479  0.002378  0.018058   
inception       Closed comedo             5.0  0.133018  0.007141  0.125600   
                Cyst                      5.0  0.017158  0.002051  0.015859   
                Dermatoglyph disruption   5.0  0.124432  0.009874  0.110877   
                Leukotrichia              5.0  0.180012  0.034299  0.161221   
                Macule                    5.0  0.099485  0.004861  0.094519   
                Nodule                    5.0  0.057494  0.004030  0.051759   
                Open comedo               5.0  0.105665  0.005552  0.100694   
                Papule                    5.0  0.114219  0.005760  0.108003   
                Patch                     5.0  0.195781  0.008334  0.185983   
                Plaque                    5.0  0.192731  0.006417  0.184046   
                Pustule                   5.0  0.030916  0.001257  0.030182   
                Scale                     5.0  0.204352  0.003816  0.199430   
                Scar                      5.0  0.214092  0.006131  0.204735   
                Sun damage                5.0  0.244161  0.016648  0.227303   
                Telangiectasia            5.0  0.065794  0.007745  0.058229   
                Thrombosed capillaries    5.0  0.030749  0.002010  0.028375   
inceptionresnet Closed comedo             5.0  0.105944  0.020036  0.089467   
                Cyst                      5.0  0.014395  0.008570  0.006186   
                Dermatoglyph disruption   5.0  0.101874  0.010459  0.091950   
                Leukotrichia              5.0  0.137783  0.035462  0.109908   
                Macule                    5.0  0.068960  0.004998  0.062544   
                Nodule                    5.0  0.043634  0.012433  0.024224   
                Open comedo               5.0  0.091292  0.002602  0.086989   
                Papule                    5.0  0.099200  0.008861  0.084756   
                Patch                     5.0  0.167383  0.008139  0.159039   
                Plaque                    5.0  0.155643  0.008075  0.143629   
                Pustule                   5.0  0.024869  0.001542  0.022878   
                Scale                     5.0  0.174835  0.009765  0.159338   
                Scar                      5.0  0.147647  0.008375  0.138680   
                Sun damage                5.0  0.211716  0.014391  0.190668   
                Telangiectasia            5.0  0.037634  0.004931  0.032049   
                Thrombosed capillaries    5.0  0.042198  0.006654  0.032079   
mobilenetv1  

In [27]:
per_characteristic_models_performance_df.reset_index().groupby(['level_0', 'level_2']).describe().head()

,level_0,level_1,level_2,iou,dice,precision,recall,negative_predictive_value,specificity
0,efficientnet,"efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0....",Closed comedo,0.130097,0.194454,0.182794,0.515469,0.947229,0.775624
1,efficientnet,"efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0....",Cyst,0.011614,0.022671,0.011777,0.407686,0.993892,0.773924
2,efficientnet,"efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0....",Dermatoglyph disruption,0.078839,0.139976,0.110504,0.317177,0.916884,0.766871
3,efficientnet,"efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0....",Leukotrichia,0.075889,0.138033,0.129896,0.166856,0.887783,0.832878
4,efficientnet,"efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0....",Macule,0.088169,0.142259,0.152398,0.351562,0.916076,0.796070
5,efficientnet,"efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0....",Nodule,0.042670,0.077886,0.059573,0.479364,0.962168,0.763142
6,efficientnet,"efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0....",Open comedo,0.120920,0.184786,0.168119,0.500477,0.940247,0.783924
7,efficientnet,"efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0....",Papule,0.108179,0.168535,0.132388,0.514348,0.961741,0.775784
8,efficientnet,"efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0....",Patch,0.174512,0.282380,0.316688,0.363363,0.827226,0.805151
9,efficientnet,"efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0....",Plaque,0.147399,0.241613,0.258154,0.381147,0.854913,0.783848


## Per derm

In [32]:
def get_per_derm_metrics(results_path, model_name, images_path, masks_path, classes, columns, derms):
    gradcams_path = results_path / 'visualisation/gradcam' / model_name
    model_path = results_path / model_name
    
    derm_mask_paths = [mask_path for mask_path in masks_path.iterdir() if mask_path.suffix == '.png']
    image_paths = [image_path for image_path in images_path.rglob('*.jpeg')]

    # For each image in the test set, calculate the value of the defined metrics given the GradCam image for a given
    # class and the outline made by a specific derm for a given characteristic. The result is stored in a defaultdict.
    # As a sanity check, the number of matches between GradCam images and derm annotations is also calculated.
    metrics_list = []

    # Set the method used for interpolation when resizing the derm annotations.
    interpolation = Image.NEAREST

    # Compute metrics for each image
    for image_path in image_paths:
        for derm in derms:
            for diagnosis in classes:
                # Check if matching GradCam file exists.
                gradcam_path = gradcams_path / Path(image_path.stem + '_' + diagnosis + '.npy')
                if gradcam_path.is_file():
                    mask_path = masks_path / Path(f'{derm}_{image_path.stem}.png') 
                    if mask_path.is_file():
                        # Calculate the value of the metics given the GradCam image and the derm mask.
                        gradcam_metric_val = calculate_mask_metrics(gradcam_path, mask_path, interpolation)
                        gradcam_metric_val['gradcam_class'] = diagnosis 
                        gradcam_metric_val['derm'] = derm
                        gradcam_metric_val['filename'] = image_path.stem 
                        metrics_list.append(gradcam_metric_val)
                else:
                    print(f'GradCam file missing for image {image_path}')
                
    return pd.DataFrame.from_records(metrics_list, columns=columns)


In [33]:
def analyse_per_derms_gradcam(
    results_path, 
    model_name, 
    images_path, 
    masks_path, 
    preds_name,
    columns,
    classes,
    derms,
    metrics
):
    per_image_metrics_df = get_per_derm_metrics(results_path, model_name, images_path, masks_path, classes, columns, derms)
    preds_df = pd.read_csv(results_path / preds_name)
    preds_df['filename'] = [filename.split('/')[1].split('.')[0] for filename in preds_df.filename.values]
    preds_df = preds_df.merge(per_image_metrics_df, left_on='filename', right_on='filename')
    # Keep only correct predictions
    preds_df = preds_df[preds_df['actual'] == preds_df['pred']]
    preds_df = preds_df[preds_df['pred_class'] == preds_df['gradcam_class']]
    metrics_dict = {}
    for derm in derms:
        metrics_dict[derm] = {}
        for metric in metrics:
            metrics_dict[derm][metric] = preds_df[preds_df['derm'] == derm][metric].mean()
            
    return metrics_dict


In [ ]:
model_info = [
    {
        'model_name': 'efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_',
        'model_architecture': 'efficientnet',
    },
    {
        'model_name': 'inception_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.001_lactivation_85_',
        'model_architecture': 'inception',
    },
    {
        'model_name': 'inceptionresnetv2_r20_s0.25_z0.5_b[0.75, 1.25]_lr0.0001_lblock8_9_ac_',
        'model_architecture': 'inceptionresnet',
    },
    {
        'model_name': 'mobilenetv1_r10_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv_pw_12_relu_',
        'model_architecture': 'mobilenetv1',
    },
    {
        'model_name': 'mobilenetv2_r10_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock_15_add_',
        'model_architecture': 'mobilenetv2',
    },
    {
        'model_name': 'nasnetmobile_r20_s0.25_z0.5_b[0.5, 1.0]_lr0.0001_lnormal_concat_11_',
        'model_architecture': 'nasnetmobile',
    },
    {
        'model_name': 'resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_',
        'model_architecture': 'resnet',
    },
    {
        'model_name': 'resnetv2_r20_s0.25_z0.25_b[0.5, 1.0]_lr0.001_lpost_relu_',
        'model_architecture': 'resnetv2',
    },
    {
        'model_name': 'vgg_r10_s0.0_z0.25_b[0.5, 1.0]_lr0.01_lblock5_pool_',
        'model_architecture': 'vgg',
    },
    {
        'model_name': 'xception_r10_s0.25_z0.5_b[0.5, 1.5]_lr0.001_lblock14_sepconv2_act_', 
        'model_architecture': 'xception',
    }
]
masks_path = Path('/home/ubuntu/hot-store/dermx_masks/per_derm')
columns = [
    'filename',
    'gradcam_class',
    'derm',
    'iou',
    'dice',
    'precision',
    'recall',
    'negative_predictive_value',
    'specificity'
]
metrics = [
    'iou',
    'dice',
    'precision',
    'recall',
    'negative_predictive_value',
    'specificity'
]

models_performance_dict = {}
for model in model_info:
    for idx in range(5):
        model_results_path = results_path / model['model_architecture']
        model_name = f'{model["model_name"]}{idx}_finetuned'
        preds_name = f'{model["model_name"]}{idx}_finetuned_preds.csv'
        print(model_name)
        model_performance = analyse_per_derms_gradcam(model_results_path, model_name, images_path, masks_path, preds_name, columns, classes, derms, metrics)

        for k, v in model_performance.items():
            models_performance_dict[(model['model_architecture'], model_name, k)] = v

per_derm_models_performance_df = pd.DataFrame(models_performance_dict).T

efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_0_finetuned


/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: invalid value encountered in double_scalars


efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_1_finetuned


/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: invalid value encountered in double_scalars


efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_2_finetuned


...

/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: invalid value encountered in double_scalars


xception_r10_s0.25_z0.5_b[0.5, 1.5]_lr0.001_lblock14_sepconv2_act_0_finetuned


/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: invalid value encountered in double_scalars


xception_r10_s0.25_z0.5_b[0.5, 1.5]_lr0.001_lblock14_sepconv2_act_1_finetuned


/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: invalid value encountered in double_scalars


In [ ]:
per_derm_models_performance_df.reset_index().groupby(['level_0', 'level_2']).describe().to_csv('per_derm_models_performance.csv')

In [ ]:
per_derm_models_performance_df.reset_index().groupby(['level_0', 'level_2']).describe()


### Old

In [2]:
# Set paths

# Path to the GradCam images.
gradcam_main_path = Path('/home/ubuntu/store/efficientnet-final-size/visualisation/gradcam/')

# Set path to the model used to create the visualisations.
model_path = Path('efficientnetb4_0')

# Path to the masks create by the derms for the individual characteristics.
derm_mask_main_path = Path('/home/ubuntu/store/masks/masks_resized')

# Path to the rescaled test images.
test_im_path = Path('/home/ubuntu/store/DermX-test-set/test/')


In [3]:
full_gradcam_path = gradcam_main_path / model_path
full_lime_path = lime_main_path / model_path

In [4]:
classes = [
    "Acne",
    "Actinic keratosis",
    "Psoriasis",
    "Seborrheic dermatitis",
    "Viral warts",
    "Vitiligo"
]

derms = [
    'derm0',
    'derm1',
    'derm2',
    'derm3',
    'derm4',
    'derm5',
    'derm6',
    'derm7',
]

chars = [
    'closed-comedo',
    'cyst',
    'dermatoglyph-disruption',
    'leukotrichia',
    'macule',
    'nodule',
    'open-comedo',
    'papule',
    'patch',
    'plaque',
    'pustule',
    'scale',
    'scar',
    'sun-damage',
    'telangiectasia',
    'thrombosed-capillaries'
]


In [5]:
# Extract the paths to the derm masks. The resulting list is only used for a sanity check.
derm_mask_paths = [p for p in derm_mask_main_path.iterdir() if p.suffix == '.png']
derm_mask_paths[0:5]

[PosixPath('/home/ubuntu/store/masks/masks_spinaltap/043023HB_mihaela_2021-05-27-masks_plaque.png'),
 PosixPath('/home/ubuntu/store/masks/masks_spinaltap/043212HB_adelina_2021-05-27-masks_scale.png'),
 PosixPath('/home/ubuntu/store/masks/masks_spinaltap/043269HB_adelina_2021-05-27-masks_papule.png'),
 PosixPath('/home/ubuntu/store/masks/masks_spinaltap/293--WatermarkedWyJXYXRlcm1hcmtlZCJd_mihaela_2021-05-27-masks_scale.png'),
 PosixPath('/home/ubuntu/store/masks/masks_spinaltap/viral-wart-08--WatermarkedWyJXYXRlcm1hcmtlZCJd_oana_2021-05-27-masks_papule.png')]

In [6]:
# Extract the paths to the test images. Only needed for visualization and debugging.
test_img_paths = [p for p in Path(test_im_path).rglob('*.jpeg')]
test_img_paths[0:5]

[PosixPath('/home/ubuntu/store/DermX-test-set/test/Actinic keratosis/017359HB.jpeg'),
 PosixPath('/home/ubuntu/store/DermX-test-set/test/Actinic keratosis/476--WatermarkedWyJXYXRlcm1hcmtlZCJd.jpeg'),
 PosixPath('/home/ubuntu/store/DermX-test-set/test/Actinic keratosis/469--WatermarkedWyJXYXRlcm1hcmtlZCJd.jpeg'),
 PosixPath('/home/ubuntu/store/DermX-test-set/test/Actinic keratosis/3742--WatermarkedWyJXYXRlcm1hcmtlZCJd.jpeg'),
 PosixPath('/home/ubuntu/store/DermX-test-set/test/Actinic keratosis/3753--WatermarkedWyJXYXRlcm1hcmtlZCJd.jpeg')]

In [7]:
# Metrics supporting probabilistic segmentation maps
fuzzy_and = lambda x,y: np.minimum(x,y)
fuzzy_or = lambda x,y: np.maximum(x,y)
fuzzy_not = lambda x: 1-x

def pixel_metrics_fuzzy(y_true, y_pred):
    """
    Pixel-level metrics of segmentation accuracy following fuzzy logic operators.
    
    :param y_true: numpy.ndarray of reference segmentation, values in [0,1]
    :param y_pred: numpy.ndarray of predicted segmentation, values in [0,1]

    :return: a dictionary encoding the metrics
    """
        
    np.testing.assert_equal(y_true.shape, y_pred.shape, err_msg="Expecting \
    the reference and predicted segmentations to be of the same size.")
    
    # Check the ranges
    np.testing.assert_equal(np.logical_and(y_true >= 0, y_true <= 1).all(), True, err_msg="Expecting \
    the reference segmentations to be in the range 0 to 1.")
    np.testing.assert_equal(np.logical_and(y_pred >= 0, y_pred <= 1).all(), True, err_msg="Expecting \
    the predicted segmentations to be in the range 0 to 1.")
    
    TP = fuzzy_and(y_true, y_pred).sum()
    TN = fuzzy_and(fuzzy_not(y_true), fuzzy_not(y_pred)).sum()
    union = fuzzy_or(y_true, y_pred).sum()
    
    metrics = {}
    
    # Summary metrics
    metrics["iou"] = TP / union
    metrics["dice"] = 2 * TP / ( y_true.sum() + y_pred.sum() ) 
    
    # Positive class metrics
    metrics["precision"] = TP / y_pred.sum()
    metrics["recall"] = TP / y_true.sum()
    
    # Negative class metrics
    metrics["negative_predictive_value"] = TN / fuzzy_not(y_pred).sum()
    metrics["specificity"] = TN / fuzzy_not(y_true).sum()
    
    return metrics

In [8]:
def calc_res(gradcam_image_path, derm_char_mask_path, interpolation_method=Image.NEAREST):
    """
    Calculates the fuzzy logic metrics given the paths to a pair of input images.
    The derm mask is resized to match the size of the gradcam image.
    
    Input:
    - gradcam_image_path: Pathlib path to a gradCam image. The file is assumed to be in .npy format.
    - derm_char_mask_path: Pathlib path to a derm annotation. The file is assumed to be in a format that can
                           be opened by PIL.
    - interpolation_method: String. The method used for interpolation when resizing the derm mask. Options are
                            NEAREST, BOX, BILINEAR, HAMMING, BICUBIC, LANCZOS. Default is NEAREST.    
    """
    
    # Open images.
    gradcam_im = np.load(gradcam_image_path, allow_pickle=True)
    mask_im = Image.open(derm_char_mask_path)
    
    # Resize the derm mask if its size does not match the size of the gradcam image.
    if gradcam_im.shape != mask_im.size[::-1]:
        # Note that resize uses (cols, rows) format, while .shape is in (rows, cols) format.
        mask_im = mask_im.resize((gradcam_im.shape[::-1]), interpolation_method)
    
    # Corvert the derm mask to numpy format and normalize to [0, 1].
    mask_im = np.asarray(mask_im) / 255
  
    res = pixel_metrics_fuzzy(mask_im, gradcam_im)
    return res

In [9]:
# For each image in the test set, calculate the value of the defined metrics given the GradCam image for a given
# class and the outline made by a specific derm for a given characteristic. The result is stored in a defaultdict.
# As a sanity check, the number of matches between GradCam images and derm annotations is also calculated.
rec_dd = lambda: defaultdict(rec_dd)
out_gradcam = rec_dd()
hit_counter = 0

# Set the method used for interpolation when resizing the derm annotations.
interpolation = Image.NEAREST

for p in test_img_paths:
    for c in classes:
        # Check if matching GradCam file exists.
        gc_path = full_gradcam_path / Path(p.stem + '_' + c + '.npy')
        if gc_path.is_file():
            for d in derms:
                for ch in chars:
                    # Check if this derm has created a mask for this characteristic.
                    mask_path = derm_mask_main_path / Path(p.stem + '_' + d + '_2021-05-27-masks_' + ch + '.png') 
                    if mask_path.is_file():
                        # Calculate the value of the metics given the GradCam image and the derm mask.
                        gradcam_metric_val = calc_res(gc_path, mask_path, interpolation)
                        out_gradcam[p.stem][c][d][ch] = gradcam_metric_val
                        hit_counter += 1
        else:
            print('GradCam or LIME file missing for image: ', p)

print(hit_counter)



19578


In [12]:
# We expect that the value of the hit counter should be equal to the number of derm masks multiplied with the number
# of classes.
if len(derm_mask_paths)*6 != hit_counter:
    print('Oh no, some files were not found. Expected/found: ', len(derm_mask_paths)*6, hit_counter)
else:
    print('GradCam images found for all derm annotations.')


GradCam images found for all derm annotations.


In [13]:
def transform_to_list(nested_dict):
    # Transform the gradcam defaultdict to a list of tuples.
    
    out = []
    for im_name, class_dict in nested_dict.items():
        for class_name, derm_dict in class_dict.items():
            for derm_name, char_dict in derm_dict.items():
                for char_name, metric
                _dict in char_dict.items():
                    tmp = tuple(metric_dict.values())
                    out.append( (im_name, class_name, derm_name, char_name) + tmp )
    return out

In [14]:
gradcam_res_list = transform_to_list(out_gradcam)
len(gradcam_res_list)

19578

## Make output dataFrames/csv files

In [18]:
col_names = ['image_name',
             'visualisation_class',
             'derm',
             'characteristic',
             'iou',
             'dice',
             'precision',
             'recall',
             'negative_predictive_value',
             'specificity'
            ]

In [ ]:
gradcam_df = pd.DataFrame.from_records(gradcam_res_list, columns=col_names)
gradcam_df

In [22]:
# Save dataFrames
model_name = str(model_path)
gradcam_df.to_csv(model_name + "_gradcam_scores.csv")


# Filter

In [123]:
import glob
model_names = glob.glob('/home/ubuntu/store/efficientnet-final-size/*h5')

In [124]:
model_names

['/home/ubuntu/store/efficientnet-final-size/efficientnetb4_4.h5',
 '/home/ubuntu/store/efficientnet-final-size/efficientnetb4_0.h5',
 '/home/ubuntu/store/efficientnet-final-size/efficientnetb4_2.h5',
 '/home/ubuntu/store/efficientnet-final-size/efficientnetb4_3.h5',
 '/home/ubuntu/store/efficientnet-final-size/efficientnetb4_1.h5']

In [125]:
model_name = Path(model_names[1]).stem

In [126]:
def get_table(results_pred):
    results_mean = results_pred.groupby('characteristic').median()
    results_std =  results_pred.groupby('characteristic').mad()
    
    ### Add means
    results_mean.loc['mean'] = results_mean.mean()
    results_std.loc['mean']   = results_mean.mean()
    
    columns = results_mean.columns.to_list()

    table_pred = results_mean
    return table_pred

class_map = {'0' : 'Acne' ,
             '1' : 'Actinic keratosis',
             '2' : 'Psoriasis' ,
             '3' : 'Seborrheic dermatitis',
             '4' : 'Viral warts',
             '5' : 'Vitiligo'}

filefolder = model_name + "_gradcam_scores.csv"
predsfile = '/home/ubuntu/store/efficientnet-final-size/' + model_name + '_preds.csv'

In [127]:
filtered_subjects = pd.read_csv('./include_images_525.csv')
filtered_subjects['image_id'] = filtered_subjects['image_id'].apply(lambda x: Path(x).stem)
filtered_subjects = filtered_subjects.rename(columns={'image_id': 'image_name'})
filtered_subjects = filtered_subjects.drop(columns=['Unnamed: 0'])

In [128]:
df_preds = pd.read_pickle(predsfile)
gradcam_df = pd.read_csv(filefolder)
gradcam_df=gradcam_df.drop(['Unnamed: 0'], axis=1)

In [129]:
df_preds['pred'] =  df_preds['pred'].apply(lambda x: class_map[str(x)])
df_preds['actual'] = df_preds['actual'].apply(lambda x: class_map[str(x)])
df_preds['filenames'] = df_preds['filenames'].apply(lambda x: Path(x).stem)
df_preds = df_preds.rename(columns={'filenames':'image_name'})
df_preds = df_preds.merge(filtered_subjects, on = 'image_name')

In [32]:
result = pd.merge(gradcam_df,df_preds, on = 'image_name')

results_pred = result[result.visualisation_class == result.pred]
table_pred = get_table(results_pred)
# Get it for actual
results_actual = result[result.visualisation_class == result.actual  ]
table_actual = get_table(results_actual)

table_pred.to_pickle('./' + model_name + '_gradcam_visualisation_scores_pred.pkl')
table_actual.to_pickle('./' + model_name + '_gradcam_visualisation_scores_actual.pkl')

results_equal = result[ (result.actual == result.pred) & (result.visualisation_class == result.pred) ]
table_equal = get_table(results_equal)

results_diff = result[ (result.actual != result.pred) & (result.visualisation_class == result.pred) ]
table_diff = get_table(results_diff)

table_equal.to_pickle('./' + model_name + '_gradcam_visualisation_scores_equal.pkl')
table_diff.to_pickle('./' + model_name + '_gradcam_visualisation_scores_diff.pkl')

# Creating Benchmarks

## By concatenation of all models

In [89]:
def lt_format(results_performance, col_n):
    results_mean = results_performance.groupby(col_n).mean()
    results_std = results_performance.groupby(col_n).std()
    
    results_mean.loc['mean'] = results_mean.mean()
    results_std.loc['mean'] = results_std.mean()
    for col in results_performance.columns.to_list():
        results_mean[col] = results_mean[col].apply(lambda x: f'{np.round(x,decimals=2)} $\pm$ ')
        results_std[col] = results_std[col].apply(lambda x: f'{np.round(x,decimals=2)}')
    
    return results_mean + results_std

model_names = glob.glob('/home/ubuntu/store/efficientnet-final-size/*h5')
results_actual = pd.DataFrame()
results_pred = pd.DataFrame()
results_equal = pd.DataFrame()
results_diff = pd.DataFrame()
results_performance = pd.DataFrame()

for mn in sorted(model_names):
    model_name = Path(mn).stem
    performance = pd.read_pickle('./' + model_name + '_performance.pkl')
    actual = pd.read_pickle('./' + model_name + '_gradcam_visualisation_scores_actual.pkl').loc[:,['recall','specificity','dice']]
    pred = pd.read_pickle('./' + model_name + '_gradcam_visualisation_scores_pred.pkl').loc[:,['recall','specificity','dice']]
    equal = pd.read_pickle('./' + model_name + '_gradcam_visualisation_scores_equal.pkl').loc[:,['recall','specificity','dice']]
    diff = pd.read_pickle('./' + model_name + '_gradcam_visualisation_scores_diff.pkl').loc[:,['recall','specificity','dice']]
    
    performance=performance.drop('mean')
    actual = actual.drop('mean')
    pred = pred.drop('mean')
    equal = equal.drop('mean')
    diff = diff.drop('mean')
    
    results_performance = pd.concat([results_performance, performance])
    results_actual = pd.concat([results_actual, actual])
    results_pred = pd.concat([results_pred, pred]) 
    results_diff = pd.concat([results_diff, diff])

results_performance.index.name = 'label'

table_performance = lt_format(results_performance, 'label')
table_actual = lt_format(results_actual, 'characteristic')

In [90]:
table_performance = table_performance.drop(columns = ['accuracy'])

In [91]:
table_performance

,precision,f1-score,specificity
label,,,
acne,0.77 $\pm$ 0.16,0.65 $\pm$ 0.32,0.97 $\pm$ 0.02
actinic_keratosis,0.74 $\pm$ 0.08,0.55 $\pm$ 0.11,0.96 $\pm$ 0.02
psoriasis_no_pustular,0.43 $\pm$ 0.14,0.57 $\pm$ 0.12,0.67 $\pm$ 0.23
seborrheic_dermatitis,0.62 $\pm$ 0.08,0.45 $\pm$ 0.22,0.95 $\pm$ 0.03
vitiligo,0.0 $\pm$ 0.01,0.0 $\pm$ 0.01,0.9 $\pm$ 0.03
wart,0.08 $\pm$ 0.04,0.07 $\pm$ 0.04,0.86 $\pm$ 0.04
mean,0.44 $\pm$ 0.08,0.38 $\pm$ 0.14,0.89 $\pm$ 0.06


In [87]:
print(table_performance.to_latex())

\begin{tabular}{llll}
\toprule
{} &        precision &         f1-score &      specificity \\
label                 &                  &                  &                  \\
\midrule
acne                  &  0.77 \$\textbackslash pm\$ 0.16 &  0.65 \$\textbackslash pm\$ 0.32 &  0.97 \$\textbackslash pm\$ 0.02 \\
actinic\_keratosis     &  0.74 \$\textbackslash pm\$ 0.08 &  0.55 \$\textbackslash pm\$ 0.11 &  0.96 \$\textbackslash pm\$ 0.02 \\
psoriasis\_no\_pustular &  0.43 \$\textbackslash pm\$ 0.14 &  0.57 \$\textbackslash pm\$ 0.12 &  0.67 \$\textbackslash pm\$ 0.23 \\
seborrheic\_dermatitis &  0.62 \$\textbackslash pm\$ 0.08 &  0.45 \$\textbackslash pm\$ 0.22 &  0.95 \$\textbackslash pm\$ 0.03 \\
vitiligo              &   0.0 \$\textbackslash pm\$ 0.01 &   0.0 \$\textbackslash pm\$ 0.01 &   0.9 \$\textbackslash pm\$ 0.03 \\
wart                  &  0.08 \$\textbackslash pm\$ 0.04 &  0.07 \$\textbackslash pm\$ 0.04 &  0.86 \$\textbackslash pm\$ 0.04 \\
mean                  &  0.44 \

In [39]:
table_actual

,recall,specificity,dice
characteristic,,,
closed-comedo,0.21 $\pm$ 0.1,0.83 $\pm$ 0.04,0.08 $\pm$ 0.03
cyst,0.2 $\pm$ 0.14,0.85 $\pm$ 0.03,0.02 $\pm$ 0.01
dermatoglyph-disruption,0.09 $\pm$ 0.05,0.79 $\pm$ 0.06,0.05 $\pm$ 0.02
leukotrichia,0.14 $\pm$ 0.1,0.82 $\pm$ 0.04,0.07 $\pm$ 0.03
macule,0.27 $\pm$ 0.07,0.8 $\pm$ 0.03,0.09 $\pm$ 0.03
nodule,0.2 $\pm$ 0.08,0.82 $\pm$ 0.03,0.03 $\pm$ 0.01
open-comedo,0.19 $\pm$ 0.1,0.83 $\pm$ 0.04,0.08 $\pm$ 0.04
papule,0.23 $\pm$ 0.07,0.8 $\pm$ 0.03,0.07 $\pm$ 0.01
patch,0.28 $\pm$ 0.06,0.8 $\pm$ 0.03,0.21 $\pm$ 0.04


In [40]:
print(table_actual[['dice','recall','specificity']].to_latex())

\begin{tabular}{llll}
\toprule
{} &             dice &           recall &      specificity \\
characteristic          &                  &                  &                  \\
\midrule
closed-comedo           &  0.08 \$\textbackslash pm\$ 0.03 &   0.21 \$\textbackslash pm\$ 0.1 &  0.83 \$\textbackslash pm\$ 0.04 \\
cyst                    &  0.02 \$\textbackslash pm\$ 0.01 &   0.2 \$\textbackslash pm\$ 0.14 &  0.85 \$\textbackslash pm\$ 0.03 \\
dermatoglyph-disruption &  0.05 \$\textbackslash pm\$ 0.02 &  0.09 \$\textbackslash pm\$ 0.05 &  0.79 \$\textbackslash pm\$ 0.06 \\
leukotrichia            &  0.07 \$\textbackslash pm\$ 0.03 &   0.14 \$\textbackslash pm\$ 0.1 &  0.82 \$\textbackslash pm\$ 0.04 \\
macule                  &  0.09 \$\textbackslash pm\$ 0.03 &  0.27 \$\textbackslash pm\$ 0.07 &   0.8 \$\textbackslash pm\$ 0.03 \\
nodule                  &  0.03 \$\textbackslash pm\$ 0.01 &   0.2 \$\textbackslash pm\$ 0.08 &  0.82 \$\textbackslash pm\$ 0.03 \\
open-comedo          